# Problem Set 6 (Total points: 100)


### Q1. Markov Decision Process [20 points]
Imagine an MDP corresponding to playing slot machines in a casino. Suppose you start with \$20\$ cash to spend in the casino,  and you decide to play until you lose all your money or until you double your money (i.e., increase your cash to at least \$40\$). There are two slot machines you can choose to play: 1) slot machine X costs \$10\$ to play and will pay out \$20\$ with probability \$0.05\$ and will pay \$0\$ otherwise;
and \$2)\$ slot machine Y costs \$20\$ to play and will pay out \$30\$ with probability 0.01 and \$0 \$ otherwise. As you are playing, you keep choosing machine X or Y at each turn.

Write down the MDP that corresponds to the above problem. Clearly specify the state space, action space, rewards and transition probabilities. Indicate which state(s) are terminal. Assume that the discount factor γ = 1.

**Notes:** There are several valid ways to specify the MDP, so you have some flexibility in your solution. For example, rewards can take many different forms, but overall you should get a higher reward for stopping when you double your money than when you lose all your money!


states: 

    {20 cash, 10 cash, 0 cash, 30 cash, 40 cash} The terminal states are 0 cash and 40 cash.

action: 

    {play slot machine X, play slot machine Y}

Transition: 

    P(20 cash, play slot X, 30 cash) = .05 #can play X, costs 10 but can win 20 => 30 cash next state
    P(20 cash, play slot X, 10 cash) = .95 #can play X, costs 10 and not win    => 10 cash next state
    P(20 cash, play slot Y, 30 cash) = .01 #can play Y, costs 20 but can win 30 => 30 cash next state
    P(20 cash, play slot Y, 0 cash)  = .99 #can play Y, costs 20 and not win    => 0 cash: terminal state
    
    P(30 cash, play slot X, 40 cash) = .05 #can play X, costs 10 but can win 20 => 40 cash: terminal state
    P(30 cash, play slot X, 20 cash) = .95 #can play X, costs 10 and not win    => 20 cash
    P(30 cash, play slot Y, 40 cash) = .01 #can play Y, costs 20 but can win 30 => 40 cash: terminal state
    P(30 cash, play slot Y, 10 cash) = .99 #can play Y, costs 20 and not win    => 10 cash:
    
    P(10 cash, play slot X, 20 cash) = .05 #can play X, costs 10 but can win 20 => 20 cash
    P(10 cash, play slot X, 0  cash) = .95 #can play X, costs 10 and not win    => 0 cash: terminal state
    
Reward:
    
    R(20 cash, play slot X, 30 cash) = 0
    R(20 cash, play slot X, 10 cash) = 0
    R(20 cash, play slot Y, 30 cash) = 0
    R(20 cash, play slot Y, 0 cash)  = -5
    
    R(30 cash, play slot X, 40 cash) = 10
    R(30 cash, play slot X, 20 cash) = 0
    R(30 cash, play slot Y, 40 cash) = 10
    R(30 cash, play slot Y, 10 cash) = 0
    
    R(10 cash, play slot X, 20 cash) = 0
    R(10 cash, play slot X, 0  cash) = -5

## Reinforcement Learning
In the remainder of the problem set you will implement the Q-Learning Algorithm to solve the "Frozen Lake" problem.
We​ ​will​ ​use​ ​OpenAI’s​ ​gym​ ​package​ ​to​ ​develop​ ​our​ ​solution​ ​in​ ​Python.​ 

### OpenAI Gym Setup 
​Read the​ ​set-up​ ​instructions for​ ​Gym​  [here](https://gym.openai.com/docs/).​ ​The​ ​instructions​ ​also​ ​give​ ​a​ ​good​ ​overview​ ​of​ ​the​ ​API​ ​for​ ​this​ ​package,​ ​so​ ​please​ ​read through​ ​it​ ​before​ ​proceeding.​



### Frozen Lake
Instead​ ​of​ ​using​ ​CartPole,​ ​we’re​ ​going​ ​to​ ​be​ ​using​ ​the​ [​​FrozenLake](https://gym.openai.com/envs/FrozenLake-v0/) environment. Read through the code for this environment by following the Github link.​ <br>

Winter​ ​is​ ​quickly​ ​approaching,​ ​and​ ​we​ ​have​ ​to​ ​worry​ ​about​ ​navigating​ ​frozen​ ​lakes.​ ​It’s​ ​only early November,​ ​
so​ ​the​ ​lakes​ ​haven’t​ ​completely​ ​frozen​ ​and​ ​if​ ​you​ ​make​ ​the​ ​wrong​ ​step​ ​you​ ​may​ ​fall​ ​through. 
We’ll​ ​need​ ​to​ ​learn​ ​how​ ​to​ ​get​ ​to​ ​our​ ​destination​ ​when​ ​stuck​ ​on​ ​the​ ​ice,​ ​without​ ​falling​ ​in.
The​ ​lake​ ​we’re​ ​going​ ​to​ ​consider​ ​is a​ ​square​ ​lake​ ​with​ ​spots​ ​to​ ​step​ ​on​ ​in​ ​the​ ​shape​ ​of​ ​a​ ​grid.​ ​​<br>

The surface is described using a 4x4 grid like the following

        S F F F 
        F H F H
        F F F H
        H F F G

​Each​ ​spot​ ​can have​ ​one​ ​of​ ​four​ ​states:
- S:​ ​starting​ ​point.
- G:​ ​goal​ ​point.
- F:​ ​frozen​ ​spot,​ ​where​ ​it’s​ ​safe​ ​to​ ​walk.
- H:​ ​hole​ ​in​ ​the​ ​ice,​ ​where​ ​it’s​ ​not​ ​safe​ ​to​ ​walk.<br>


For example, consider the lake, <br>
![alt text](https://cs-people.bu.edu/sunxm/frozen_lake_example.png)

There are four possible actions: UP, DOWN, LEFT, RIGHT. Although we​ ​can​ ​see​ ​the​ ​path​ ​we​ ​need​ ​to​ ​walk,​ the agent does not. ​We’re​ ​going​ ​to​ ​train​ ​an​ ​agent​ ​to​ ​discover​ ​this​ ​via​ ​problem solving.​ ​However,​ ​walking​ ​on​ ​ice​ ​isn’t​ ​so​ ​easy!​ ​Sometimes​ ​you​ ​slip​ ​and​ ​aren’t​ ​able​ ​to​ ​take​ ​the​ ​step​ ​you intended.

The episode ends when you reach the goal or fall in a hole.

You receive a reward of 1 if you reach the goal, and zero otherwise.

#### Q2. Walking on the Frozen Lake [10 points]

Write a script that sets up the Frozen Lake environment and takes 10 walks through it, consisting of a maximum 10 randomly sampled actions during each walk. After each step, render the current state, and print out the reward and whether the walk is "done", i.e. in the terminal state, because the agent fell into a hole (stop if it is). In your own words, explain how this environment behaves. 

The environment is static and does not change whenever the agent takes an action. Only the state that the agent observes changes.

In [3]:
pip install gym

  Using cached gym-0.17.3.tar.gz (1.6 MB)
  Using cached pyglet-1.5.0-py2.py3-none-any.whl (1.0 MB)
  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654654 sha256=3656e67342c7828d8f45bb5e6f4a73af555ea98c77fa7f8f698fe3c44d584dbe
  Stored in directory: /Users/khoatran/Library/Caches/pip/wheels/84/40/e7/14efb9870cfc92ac236d78cb721dce614ddec9666c8a5e0a35
Successfully built gym
Note: you may need to restart the kernel to use updated packages.


In [3]:
import gym
import numpy as np
env = gym.make('FrozenLake-v0')
# env = gym.make('FrozenLake-v0').unwrapped
env.reset()
for _ in range(10):
    env.reset()
    for _ in range(10):
        env.render()
        observation, reward, done, info  = env.step(env.action_space.sample()) # take a random action
        print("Reward:",reward, "Done:", done)
        if done:
            break;
env.close()



SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Up)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Down)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Down)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: True

SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Up)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Left)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Left)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Up)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Right)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: True

SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Up)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Right)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Left)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Up)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Left)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Up)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: True

SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Up)
SFFF
FHFH
FFFH
HFFG
Reward: 0.0 Done: False
  (Right)
SFFF
FHFH
FFFH
HFFG
Reward: 0.

#### Q3. Q-Learning [50 points]

You will ​implement​ ​Q-learning to solve the problem.​ Assume that the environment has states S and actions A. Use the ​function​ ​signature​ ​provided​ below:

``` python
def​ q_learning(env,​ ​alpha=0.5,​ gamma=0.95,​ ​epsilon=0.1, num_episodes=500):
""" ​Performs​ ​Q-learning​ ​for​ ​the​ ​given​ ​environment.
Initialize​ ​Q​ ​to​ ​all​ ​zeros.
:param​ ​env:​ ​Unwrapped​ ​OpenAI​ ​gym​ ​environment.
:param​ ​alpha:​ ​Learning​ ​rate​ ​parameter.
:param​ ​gamma:​ ​Decay​ ​rate​ (future reward discount) ​parameter.
:param​ ​num_episodes:​ ​Number​ ​of​ ​episodes​ ​to​ ​use​ ​for​ ​learning. 
:return:​ ​Q​ ​table, i.e. a table with the Q value for every <S, A> pair.
"""
```


The pseudocode for Q-Learning was described in lecture, but for your reference, we provide it here:
![alt text](https://cs-people.bu.edu/sunxm/q-learning.png)


In [4]:
def q_learning(env, alpha=0.5,gamma=.95, epsilon=0.1,num_episodes=500, init_method = "Z", test = False):
    np.random.seed(42)
    
    #init method Z: zeros  R: randoms
    if init_method == "Z":
        Q = np.zeros((env.nS, env.nA))
    elif init_method == "R":
        Q = np.random.rand(env.nS, env.nA)

    for _ in range(num_episodes):
        done = False
        
        #reset state
        currState = env.reset()
        
        #decrease exploration percentage
        ep = epsilon
        if ep < .01:
            ep = .01
        else:
            ep = ep - .0001
        while not done:
            if not test:
                #epsilon greedy
                exploreProb = np.random.uniform(0,1)
                if exploreProb < ep:
                    currAction = env.action_space.sample()
                else:
                    currAction = np.argmax(Q[currState])
            else:
                #argmax
                currAction = np.argmax(Q[currState])

            #take step    
            newState, reward, done, info= env.step(currAction)
            
            #max Q of new
            max_Q_NS_NA = np.max(Q[newState])
            
            #update
            newQ = Q[currState][currAction] + (alpha * ((reward + (gamma * max_Q_NS_NA) - Q[currState][currAction])))
            Q[currState][currAction] = newQ
            
            #set curr state to new state
            currState = newState
    return Q

#### Q4. Main Function [20 points]
You also need to implement the main function to solve the entire FrozenLake-v0 problem, including setting up the Gym environment, 
calling the q-learning function as defined above, printing out the returned Q-table, etc. <br>

You should use the $\epsilon$-greedy algorithm (solving Exploration-Exploitation Dilemma) to generate actions for each state. Try `num_episodes` with different values, e.g. `num_episodes=500, 1000, 5000, 10000, 50000`. You should also try different initializations for the Q-table, i.e. Random Initialization and Zero Initialization. 

Please do not change any default value of environment setting, such as `is_slippery`, if not mentioned above.

Provide​ ​the​ ​final​ ​Q-table​ ​of​ ​​FrozenLake-v0​ for **each <num_episode, init_method>** you have tried [10 points],​ ​and analyze ​what​ you observe [10 points]. 

If you are interested in testing your learned Q-learned, you can compute the win rate (the probability of agents to goal following the Q-table). Note that you should use `argmax` instead of epsilon greedy to choose the action at the current state. 
Win Rate is a good way to test whether you write your algorithm correctly.

Just for your reference, we test our implementation for 10000 trials after 50000-episode training using zero initialization and the win rate is 0.4985. You should achieve similar value under the same test.


ANALYSIS: As the number of episodes increases, the Q table changes further. With the number of episodes at 50000, the final Q table has 0 for all actions for all the terminal states.


In [5]:
numE = [500, 1000, 5000, 10000, 50000]
initMethod = ["R", "Z"]

env = gym.make('FrozenLake-v0').unwrapped

for numEps in numE:
    for method in initMethod:
        print("number of episodes:", numEps)
        print("init method:", method)
        print(q_learning(env, .5, .95, .1, numEps, method, False))

number of episodes: 500
init method: R
[[0.5504985  0.54046344 0.55015704 0.50969783]
 [0.5685941  0.50383225 0.47406929 0.47066274]
 [0.5424954  0.48269084 0.47941828 0.48140248]
 [0.49268248 0.46699931 0.47942985 0.18340451]
 [0.59005854 0.62856359 0.55894114 0.56375093]
 [0.61185289 0.13949386 0.29214465 0.36636184]
 [0.55604442 0.58417428 0.57758893 0.51423444]
 [0.59241457 0.04645041 0.60754485 0.17052412]
 [0.69174812 0.74610109 0.60894467 0.67292518]
 [0.67945776 0.75767774 0.60085268 0.49505697]
 [0.4765239  0.78901999 0.30146265 0.62024419]
 [0.25877998 0.66252228 0.31171108 0.52006802]
 [0.54671028 0.18485446 0.96958463 0.77513282]
 [0.88483645 0.60194558 0.70741418 1.06403675]
 [0.32268537 0.19598286 0.04522729 1.35206443]
 [0.38867729 0.27134903 0.82873751 0.35675333]]
number of episodes: 500
init method: Z
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0.

#### Additional Instructions
If​ ​you’re​ ​new​ ​to​ OpenAI’s​ ​Gym,​ first ​go​ ​through​ ​OpenAI’s​ ​full​ ​tutorial listed​ ​earlier​ ​and​ ​visit​ ​the​ ​Appendix​ ​to​ ​this​ ​homework​ ​before​ ​proceeding.
Some additional rules:
- Only submit **original code** written entirely by you.
- **Permitted​​ non-standard ​​libraries**:​​ ​gym​,​​ ​numpy.
- **Only​ ​use​ ​numpy​ ​for​ ​random​ ​sampling,​ ​and​ ​seed​ ​at​ ​the​ ​beginning​ ​of​ ​each​ ​function​ ​with**:
np.random.seed(42)
- **Unwrap​ ​the​ ​OpenAI​ ​gym​ ​before​ ​providing​ ​them​ ​to​ ​these​ ​functions.** <br>
env​ ​=​ ​gym.make(“FrozenLake-v0”).unwrapped


## Appendix

This​ ​appendix​ ​includes​ ​references​ ​to​ ​APIs​ ​you​ ​may​ ​find​ ​useful.​ ​If​ ​the​ ​description​ ​sounds​ ​useful,​ ​check out
the​ ​respective​ ​package’s​ ​documentation​ ​for​ ​a​ ​much​ ​better​ ​description​ ​than​ ​we​ ​could​ ​provide.

#### Numpy
- np.zeros:​ ​N-dimensional​ ​tensor​ ​initialized​ ​to​ ​all​ ​0s.
- np.ones:​ ​N-dimensional​ ​tensor​ ​initialized​ ​to​ ​all​ ​1s.
- np.eye:​ ​N-dimensional​ ​tensor​ ​initialized​ ​to​ ​a​ ​diagonal​ ​matrix​ ​of​ ​1s.
- np.random.choice:​ ​Randomly​ ​sample​ ​from​ ​a​ ​list,​ ​allowing​ ​you​ ​to​ ​specify​ ​weights.
- np.argmax:​ ​Index​ ​of​ ​the​ ​maximum​ ​element.
- np.abs:​ ​Absolute​ ​value.
- np.mean:​ ​Average​ ​across​ ​dimensions.
- np.sum:​ ​Sum​ ​across​ ​dimensions.

### OpenAI Gym
- Environment​ ​(unwrapped):<br>
env.nS #​ ​Number​ ​of​ ​spaces. <br>
env.nA #​ ​Number​ ​of​ ​actions. <br>
env.P #​ ​Dynamics​ ​model.
